In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Mounted at /content/gdrive


In [3]:
!nvidia-smi

Sun Apr 14 18:35:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
pip install -U ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.238
    Uninstalling ultralytics-8.0.238:
      Successfully uninstalled ultralytics-8.0.238


In [4]:
!pip install --upgrade ultralytics==8.0.238 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 695.2/695.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.

In [6]:
ROOT_DIR = '/content/gdrive/My Drive/hyperpar_tuning'

In [7]:
sweep_configuration = {
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "metrics/mAP50(B)"},
    "parameters": {
        "learning_rate": {"max": 0.01, "min": 0.0001},
        "batch_size": {"values": [16, 32, 64, 128]},
        "optimizer": {"values": ["Adam", "SGD", "AdamW"]},
        "momentum": {"max": 0.98, "min": 0.6},
        "weight_decay": {"max": 0.001, "min": 0.0},
        "final_learning_rate": {"max": 1.0, "min": 0.01}
    },
}

In [8]:
sweep_configuration

{'method': 'bayes',
 'metric': {'goal': 'maximize', 'name': 'metrics/mAP50(B)'},
 'parameters': {'learning_rate': {'max': 0.01, 'min': 0.0001},
  'batch_size': {'values': [16, 32, 64, 128]},
  'optimizer': {'values': ['Adam', 'SGD', 'AdamW']},
  'momentum': {'max': 0.98, 'min': 0.6},
  'weight_decay': {'max': 0.001, 'min': 0.0},
  'final_learning_rate': {'max': 1.0, 'min': 0.01}}}

In [9]:
import wandb
from wandb.integration.ultralytics import add_wandb_callback
import os

from ultralytics import YOLO


wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [10]:
sweep_id = wandb.sweep(sweep=sweep_configuration, entity = 'mtarwacki', project="hyperparameters_mAP50"  )

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: j70ahjyq
Sweep URL: https://wandb.ai/mtarwacki/hyperparameters_mAP50/sweeps/j70ahjyq


In [11]:
def train_sweep(config = None):
  with wandb.init(config=sweep_configuration) as run:
    run.name=f"YOLOv8n_{wandb.config.learning_rate}_{wandb.config.batch_size}_{wandb.config.optimizer}"
    config = wandb.config
    model = YOLO('yolov8n.pt')
    model.train(data='/content/gdrive/My Drive/hyperpar_tuning/data.yaml', epochs=60, patience = 30, fliplr = 0.0, flipud = 0.0,
                lr0 = wandb.config.learning_rate, batch = wandb.config.batch_size, optimizer = wandb.config.optimizer,
                momentum = wandb.config.momentum, weight_decay = wandb.config.weight_decay, lrf = wandb.config.final_learning_rate)


In [12]:
wandb.agent(sweep_id = '1746kzo3', function=train_sweep, count = 19)

wandb: Agent Starting Run: emliwarb with config:
wandb: 	batch_size: 64
wandb: 	final_learning_rate: 0.3372305076696039
wandb: 	learning_rate: 0.008216016407139225
wandb: 	momentum: 0.8920492746314275
wandb: 	optimizer: AdamW
wandb: 	weight_decay: 5.9073760568619574e-05
wandb: Currently logged in as: maciej-tarwacki-stud (mtarwacki). Use `wandb login --relogin` to force relogin


100%|██████████| 6.23M/6.23M [00:00<00:00, 185MB/s]


Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/gdrive/My Drive/hyperpar_tuning/data.yaml, epochs=60, time=None, patience=30, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, sho

100%|██████████| 755k/755k [00:00<00:00, 44.5MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...

                   all        450        710      0.517       0.53      0.548       0.26



60 epochs completed in 0.512 hours.
Optimizer stripped from runs/detect/train19/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train19/weights/best.pt, 6.2MB

Validating runs/detect/train19/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:03<00:00,  3.80it/s]


                   all        450        710      0.702      0.582      0.616      0.288
                crease        450         15          1     0.0908      0.479      0.153
          crescent_gap        450         52      0.788      0.998      0.956      0.626
             inclusion        450         69      0.273      0.406      0.203      0.063
              oil_spot        450        108      0.768      0.336      0.544      0.225
         punching_hole        450         63      0.875      0.937      0.968      0.521
            rolled_pit        450         17       0.47      0.294      0.199     0.0602
             silk_spot        450        187      0.639      0.503      0.533      0.216
         waist_folding        450         28        0.7      0.751      0.782      0.337
            water_spot        450         73      0.898      0.602      0.664      0.365
          welding_line        450         98       0.61      0.898      0.837      0.311
Speed: 0.1ms preproce

lr/pg0,█▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▁▅████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆
lr/pg2,▁▅████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆
metrics/mAP50(B),▂▁▃▄▅▄▆▅▆▆▅▆▆▆▇▆▆▄▇▇▆▇▇▇▇▇▇██▇▇████▇▆█▇█
metrics/mAP50-95(B),▂▁▃▄▅▄▆▅▆▆▅▆▆▆▇▆▆▄▇▇▆▇▇▇█▇████▇███▇▇▆█▇█
metrics/precision(B),▆▁█▅▄█▅▃▄▄▃▅▄▅▅▅▄▄▅▅▆▅▅▅▅▆▅▅▆▅▅▆▆▅▅▆▄▆▅▆
metrics/recall(B),▁▁▁▄▆▃▆▆▆▇▅▇▇▆▇▆▇▄▇▇▆▇▆▇█▇▇████▇▇██▇▆█▇█
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▅▄▄▄▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▃▃▃▃▃▃


In [ ]:
!scp -r /content/runs '/content/gdrive/My Drive/hyperpar_tuning'